In [33]:
# Import block
import os
import pandas as pd
import numpy as np
import time
from math import sqrt
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pandas.plotting import scatter_matrix
from importlib import reload
from sklearn.feature_selection import VarianceThreshold
from collections import Counter
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [45]:
grenade_data_path = '../grenade.csv'
shield_data_path = '../shield.csv'
reload_data_path = '../reload.csv'
logout_data_path = '../logout.csv'
# noise_data_path = '../expected_data_noise.csv'


grenade_data_raw = pd.read_csv(grenade_data_path, index_col=False)
shield_data_raw = pd.read_csv(shield_data_path, index_col=False)
reload_data_raw = pd.read_csv(reload_data_path, index_col=False)
logout_data_raw = pd.read_csv(logout_data_path, index_col=False)
# noise_data_raw = pd.read_csv(noise_data_path, index_col=False)

grenade_data_raw = grenade_data_raw.apply(pd.to_numeric)
grenade_data_raw.reset_index(inplace=True, drop=True)

shield_data_raw = shield_data_raw.apply(pd.to_numeric)
shield_data_raw.reset_index(inplace=True, drop=True)

reload_data_raw = reload_data_raw.apply(pd.to_numeric)
reload_data_raw.reset_index(inplace=True, drop=True)
reload_data_raw = reload_data_raw.drop(index=[i for i in range(4000, 6399)])

logout_data_raw = logout_data_raw.apply(pd.to_numeric)
logout_data_raw.reset_index(inplace=True, drop=True)

# noise_data_raw = noise_data_raw.drop(1491).drop(1453).apply(pd.to_numeric)
# noise_data_raw.reset_index(inplace=True, drop=True)
logout_data_raw

,19.0,-22.2,-63.1,-518.0,-169.0,-123.0
0,16.78,-16.51,-60.09,-425.0,-153.0,-72.0
1,14.86,-10.17,-57.48,-327.0,-146.0,-5.0
2,13.21,-3.30,-55.33,-210.0,151.0,73.0
3,11.78,-3.96,-53.70,-66.0,162.0,170.0
4,10.59,-11.35,-52.60,118.0,169.0,272.0
...,...,...,...,...,...,...
4054,56.52,-41.33,-152.88,223.0,57.0,156.0
4055,56.25,-38.36,-154.73,213.0,53.0,132.0
4056,55.60,-35.70,-157.11,32.0,86.0,131.0
4057,53.77,-30.89,-160.54,259.0,53.0,135.0


In [47]:
WINDOW_SIZE = 20
SLIDE_SIZE = 20

def process_df_with_sliding_window(df, window_size, slide_size):
    attributes = ["acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z"]
    columns_list = []
    # Build column list
    for idx in range(1, 1 + window_size):
        for attribute in attributes:
            columns_list.append(attribute + '_' + str(idx))
            
    df_out = pd.DataFrame(columns=columns_list)

    for row_idx in range(0, len(df) - window_size, slide_size):
        curr_window_data = []
        nan_found = False
        for row_iter_idx in range(window_size):
            curr_row_idx = row_idx + row_iter_idx
            if df.loc[curr_row_idx, :].isnull().values.any():
                nan_found = True
                break
            curr_row_list = df.loc[curr_row_idx, :].values.flatten().tolist()
            curr_window_data += curr_row_list
        
        if nan_found:
            continue

        df_out.loc[len(df_out)] = curr_window_data


    return df_out

grenade_df_processed = process_df_with_sliding_window(grenade_data_raw, WINDOW_SIZE, SLIDE_SIZE)
shield_data_processed = process_df_with_sliding_window(shield_data_raw, WINDOW_SIZE, SLIDE_SIZE)
reload_data_processed = process_df_with_sliding_window(reload_data_raw, WINDOW_SIZE, SLIDE_SIZE)
logout_data_processed = process_df_with_sliding_window(logout_data_raw, WINDOW_SIZE, SLIDE_SIZE)
# noise_data_processed = process_df_with_sliding_window(noise_data_raw, WINDOW_SIZE, SLIDE_SIZE)
reload_data_processed


,acc_x_1,acc_y_1,acc_z_1,gyro_x_1,gyro_y_1,gyro_z_1,acc_x_2,acc_y_2,acc_z_2,gyro_x_2,...,acc_z_19,gyro_x_19,gyro_y_19,gyro_z_19,acc_x_20,acc_y_20,acc_z_20,gyro_x_20,gyro_y_20,gyro_z_20
0,-48.50,-23.28,33.17,-150.0,-234.0,606.0,-49.54,-22.03,31.81,-147.0,...,-53.48,0.0,83.0,968.0,-32.14,-23.82,32.21,-421.0,-108.0,485.0
1,-35.22,-21.86,29.71,-393.0,-71.0,480.0,-38.18,-19.71,27.19,-340.0,...,-46.08,216.0,82.0,788.0,3.37,-42.45,-80.10,-576.0,-335.0,413.0
2,-1.30,-42.10,77.21,-536.0,-287.0,427.0,-5.99,-41.36,74.19,-479.0,...,-67.35,126.0,19.0,910.0,14.58,-11.42,-9.47,529.0,737.0,842.0
3,18.18,-12.88,-6.76,416.0,805.0,856.0,21.71,-14.99,-4.05,280.0,...,9.62,-50.0,-248.0,602.0,12.57,-23.53,-18.19,853.0,338.0,858.0
4,16.13,-27.36,-15.57,813.0,416.0,889.0,20.08,-30.92,-12.68,782.0,...,22.98,-45.0,-253.0,495.0,19.99,-15.05,-11.02,781.0,406.0,866.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,24.04,-14.50,-63.43,-121.0,-257.0,553.0,23.08,-17.24,-67.67,-188.0,...,98.06,-86.0,-193.0,537.0,15.99,-13.83,-65.81,949.0,618.0,726.0
195,17.83,-13.24,-61.52,803.0,627.0,631.0,19.63,-12.86,-57.18,651.0,...,-130.33,719.0,874.0,21.0,6.99,-4.70,-72.29,928.0,411.0,532.0
196,8.81,-4.53,-68.77,859.0,387.0,483.0,12.65,-4.24,-61.53,774.0,...,-129.78,57.0,289.0,197.0,5.58,0.36,76.61,959.0,505.0,654.0
197,7.51,1.00,73.22,908.0,520.0,627.0,9.61,1.69,69.67,841.0,...,-127.55,465.0,138.0,239.0,4.62,13.10,92.06,898.0,452.0,107.0


In [48]:
from sklearn.preprocessing import normalize

# combined_data_features = pd.concat([grenade_df_processed, shield_data_processed, reload_data_processed, logout_data_processed, noise_data_processed])
combined_data_features = pd.concat([grenade_df_processed, shield_data_processed, reload_data_processed, logout_data_processed])
combined_data_features.reset_index(inplace=True, drop=True)
# combined_data_features = combined_data_features.apply(pd.to_numeric)

# for _, i in combined_data_features.iterrows():

#     for c in range(60):
#         if type(i[c]) != float and type(i[c]) != str:
#             print(i[c], type(i[c]))

In [49]:
# Build label
combined_data_labels = pd.DataFrame(columns=['Activity'])

for i in range(len(grenade_df_processed)):
    combined_data_labels.loc[len(combined_data_labels)] = 0
    
for i in range(len(shield_data_processed)):
    combined_data_labels.loc[len(combined_data_labels)] = 1
    
for i in range(len(reload_data_processed)):
    combined_data_labels.loc[len(combined_data_labels)] = 2
    
for i in range(len(logout_data_processed)):
    combined_data_labels.loc[len(combined_data_labels)] = 3
    
# for i in range(len(noise_data_processed)):
#     combined_data_labels.loc[len(combined_data_labels)] = 4
    
combined_data_labels

,Activity
0,0
1,0
2,0
3,0
4,0
...,...
872,3
873,3
874,3
875,3


In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_data_features, combined_data_labels, test_size=0.1, random_state=42)

X_train

,acc_x_1,acc_y_1,acc_z_1,gyro_x_1,gyro_y_1,gyro_z_1,acc_x_2,acc_y_2,acc_z_2,gyro_x_2,...,acc_z_19,gyro_x_19,gyro_y_19,gyro_z_19,acc_x_20,acc_y_20,acc_z_20,gyro_x_20,gyro_y_20,gyro_z_20
862,7.89,39.99,108.22,-570.0,87.0,228.0,9.36,42.65,106.60,-779.0,...,129.52,-149.0,152.0,27.0,4.22,38.96,120.23,-253.0,80.0,148.0
650,2.27,16.07,69.26,244.0,724.0,325.0,4.32,16.43,64.84,194.0,...,-117.37,58.0,335.0,436.0,9.29,15.52,62.26,131.0,826.0,372.0
309,62.64,9.63,14.86,-7.0,-231.0,487.0,64.34,7.29,15.95,-75.0,...,36.46,-290.0,-776.0,402.0,5.39,-20.38,-37.87,-208.0,-597.0,449.0
457,-3.53,-62.02,101.32,76.0,243.0,866.0,2.36,-63.43,-103.34,73.0,...,-131.48,-356.0,-85.0,545.0,6.95,-45.69,-128.29,269.0,367.0,836.0
473,1.96,49.12,63.44,452.0,286.0,860.0,2.30,51.01,62.21,421.0,...,-53.04,-281.0,-968.0,248.0,18.07,60.80,66.02,39.0,41.0,829.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-5.04,-1.19,174.21,-463.0,-1.0,315.0,-5.72,-3.52,175.95,-526.0,...,-1.86,-733.0,-674.0,618.0,-10.17,-10.89,173.05,-525.0,-45.0,314.0
270,-10.44,-52.20,79.63,-166.0,288.0,425.0,-13.81,-48.83,76.87,-206.0,...,54.26,-262.0,-209.0,79.0,7.94,-2.99,-137.53,88.0,486.0,414.0
860,5.64,13.91,106.29,-299.0,4.0,138.0,4.04,6.27,108.07,-95.0,...,121.86,-62.0,-51.0,-61.0,7.90,34.62,114.78,-296.0,98.0,279.0
435,8.96,56.00,91.49,234.0,157.0,500.0,5.19,58.43,89.14,192.0,...,-33.65,-175.0,-919.0,61.0,1.25,49.51,81.19,459.0,263.0,505.0


In [51]:
def normalize_data(df):
    for col in df.columns:
        print("Maximum obtained: ", df[col].abs().max())
        df[col] = df[col] / df[col].abs().max()
    
    return df

X_train = normalize_data(X_train)
X_test = normalize_data(X_test)

Maximum obtained:  106.7
Maximum obtained:  106.52
Maximum obtained:  179.55
Maximum obtained:  997.0
Maximum obtained:  995.0
Maximum obtained:  996.0
Maximum obtained:  115.37
Maximum obtained:  106.37
Maximum obtained:  179.81
Maximum obtained:  996.0
Maximum obtained:  975.0
Maximum obtained:  993.0
Maximum obtained:  123.62
Maximum obtained:  106.48
Maximum obtained:  179.95
Maximum obtained:  999.0
Maximum obtained:  995.0
Maximum obtained:  998.0
Maximum obtained:  131.5
Maximum obtained:  106.86
Maximum obtained:  179.91
Maximum obtained:  993.0
Maximum obtained:  984.0
Maximum obtained:  999.0
Maximum obtained:  139.29
Maximum obtained:  163.25
Maximum obtained:  179.86
Maximum obtained:  997.0
Maximum obtained:  985.0
Maximum obtained:  995.0
Maximum obtained:  153.57
Maximum obtained:  163.68
Maximum obtained:  179.63
Maximum obtained:  995.0
Maximum obtained:  986.0
Maximum obtained:  994.0
Maximum obtained:  171.36
Maximum obtained:  164.24
Maximum obtained:  179.48
Maximu

In [52]:
X_train.dtypes

acc_x_1      float64
acc_y_1      float64
acc_z_1      float64
gyro_x_1     float64
gyro_y_1     float64
              ...   
acc_y_20     float64
acc_z_20     float64
gyro_x_20    float64
gyro_y_20    float64
gyro_z_20    float64
Length: 120, dtype: object

In [60]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes = (120), max_iter=180)

mlp.fit(X_train, y_train)

print(mlp.score(X_train, y_train))

mlp.score(X_test, y_test)


/Users/bryanwong/Documents/laser-tag/classifier/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1096: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1.0


0.9886363636363636

In [61]:
for idx in range(len(mlp.coefs_)):
    weights = mlp.coefs_[idx].tolist()
    cpp_file = [
        'double weights', str(idx), '[', str(len(weights)), '][', str(len(weights[0])), '] = {\n'
    ]
    for row in range(len(weights)):
        for col in range(len(weights[0])):
            weights[row][col] = str(round(weights[row][col], 4))
    for index, row_weights in enumerate(weights):
        cpp_file.append('{')
        output_str = ', '.join(row_weights)
        cpp_file.append(output_str)
        append_str = "},\n"
        cpp_file.append(append_str)
    cpp_file.append('};')

    cpp_program_string = ''.join(cpp_file)
    with open('/Users/bryanwong/Documents/classifier/weights' + str(idx) + '.cpp', 'w') as f:
        f.write(cpp_program_string)
        f.close()
        
for idx in range(len(mlp.intercepts_)):
    biases = mlp.intercepts_[idx].tolist()
    cpp_file = [
        'double biases', str(idx), '[1][', str(len(biases)), '] = {'
    ]
    for col in range(len(biases)):
        biases[col] = str(round(biases[col], 4))
    output_str = ', '.join(biases)
    output_str = "{" + output_str + "}};\n"
    cpp_file.append(output_str)

    cpp_program_string = ''.join(cpp_file)
    with open('/Users/bryanwong/Documents/classifier/biases' + str(idx) + '.cpp', 'w') as f:
        f.write(cpp_program_string)
        f.close()

In [11]:
input_param = X_train.iloc[1000].tolist()
for row in range(len(input_param)):
    input_param[row] = str(round(input_param[row], 2))
output_str = ', '.join(input_param)
output_str = "test_case = [" + output_str + "]"
print(y_train.iloc[1000])
output_str


Activity    2
Name: 2765, dtype: int64


'test_case = [-0.05, 0.05, 0.25, 0.13, 0.41, 0.78, -0.04, 0.06, 0.23, 0.89, 0.82, 0.36, -0.03, 0.08, 0.21, 0.25, 0.29, 0.16, -0.02, 0.11, 0.18, 0.59, 0.84, 0.98, -0.0, 0.14, 0.14, 0.13, 0.3, 0.43, 0.02, -0.18, -0.11, 0.43, 0.76, 0.77, 0.05, -0.21, -0.07, 0.42, 0.18, 0.13, 0.1, -0.25, -0.03, 0.37, 0.33, 0.64, 0.14, -0.29, -0.02, 0.6, 0.43, 0.29, 0.23, -0.33, -0.07, 0.61, 0.29, 0.1, 0.28, -0.36, -0.13, 0.71, 0.96, 0.02, 0.35, -0.37, -0.17, 0.18, 0.55, 0.92, 0.41, -0.37, -0.2, 0.44, 0.86, 0.61, 0.44, -0.37, -0.22, 0.03, 0.13, 0.05, 0.46, -0.33, -0.22, 0.71, 0.36, 0.3, 0.47, -0.31, -0.22, -0.3, -0.25, 0.43, 0.47, -0.3, -0.21, -0.86, -0.64, 0.43, 0.47, -0.29, -0.19, -0.72, -0.2, 0.35, 0.47, -0.29, -0.18, -0.33, -0.57, 0.24, 0.46, -0.28, -0.16, -0.79, -0.9, 0.09]'

In [12]:
mlp.out_activation_

'softmax'